In [2]:
# importing old libraries (also used in the two first parts)
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs

In [2]:
# importing new libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Load the pandas dataframe (created in part 2 of the assignment)

In [3]:
df_toronto = pd.read_csv('toronto_base.csv')
df_toronto.head(7)

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
1,1,M4G,East York,Leaside,43.709060,-79.363452
2,2,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
3,3,M1W,Scarborough,Steeles West,43.799525,-79.318389
4,4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
5,5,M9P,Etobicoke,Westmount,43.696319,-79.532242
6,6,M1G,Scarborough,Woburn,43.770992,-79.216917


### Create a map of Toronto

### Create a new data frame with neighborhoods in Scarborough 

In [16]:
# @hiddel_cell
CLIENT_ID = '0MJA3NYYG3U2ZY1LTZN2OYEHS3Y3WVSON2GBSO3IL4EDYVIR' # your Foursquare ID
CLIENT_SECRET = 'WGWSAF2TKVUQPE3PD0N3EOITFVBY5EYP1VCZI3BMUG0ROUS5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



In [17]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,3,M1W,Scarborough,Steeles West,43.799525,-79.318389
1,4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
2,6,M1G,Scarborough,Woburn,43.770992,-79.216917
3,10,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
4,16,M1N,Scarborough,Birch Cliff,43.692657,-79.264848
5,31,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
6,34,M1V,Scarborough,"Agincourt North, Milliken",43.815252,-79.284577


### Create a map of Scarborough and its neighbourhoods

In [18]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


### Get the top 100 venues in the neighborhood 'Steeles West', from Scarborough

In [20]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Steeles West" are 43.799525200000005, -79.3183887.


In [21]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5b91c580f594df37694226ec'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 43,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLa

In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney store,Toy / Game Store,43.775537,-79.256833
1,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
2,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
3,DAVIDsTEA,Tea Room,43.776613,-79.258516
4,SEPHORA,Cosmetics Shop,43.775592,-79.258242
5,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,Hot Topic,Clothing Store,43.775450,-79.257929
8,Scarborough Town Centre,Shopping Mall,43.775231,-79.257462
9,American Eagle Outfitters,Clothing Store,43.775908,-79.258352


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

43 venues were returned by Foursquare.


In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get venues for each neighborhood in Scarborough

In [30]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Steeles West
Scarborough Village
Woburn
Highland Creek, Rouge Hill, Port Union
Birch Cliff
Maryvale, Wexford
Agincourt North, Milliken
Cedarbrae
Tam O'Shanter
Cliffcrest, Cliffside
Morningside, West Hill
Rouge, Malvern
Agincourt
Ionview, Kennedy Park
Dorset Park, Scarborough Town Centre, Wexford Heights
Upper Rouge
Clairlea, Golden Mile, Oakridge


In [34]:
scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Steeles West,43.799525,-79.318389,Mr Congee Chinese Cuisine 龍粥記,43.798879,-79.318335,Chinese Restaurant
1,Steeles West,43.799525,-79.318389,Phoenix Restaurant 金鳳餐廳,43.798198,-79.318432,Chinese Restaurant
2,Steeles West,43.799525,-79.318389,Price Chopper,43.799445,-79.318563,Grocery Store
3,Steeles West,43.799525,-79.318389,Subway,43.798983,-79.318838,Sandwich Place
4,Steeles West,43.799525,-79.318389,Shoppers Drug Mart,43.799670,-79.319315,Pharmacy
5,Steeles West,43.799525,-79.318389,Tim Hortons,43.798281,-79.318317,Coffee Shop
6,Steeles West,43.799525,-79.318389,KFC,43.798938,-79.318854,Fast Food Restaurant
7,Steeles West,43.799525,-79.318389,Yamamoto Japanese Cuisine 山本盛世,43.798589,-79.318558,Japanese Restaurant
8,Steeles West,43.799525,-79.318389,McDonald's,43.798880,-79.318724,Fast Food Restaurant
9,Steeles West,43.799525,-79.318389,Pizza Pizza,43.797909,-79.318113,Pizza Place


In [35]:
scarborough_venues.tail(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
79,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Jerry's Fish & Chips,43.753717,-79.276784,Fish & Chips Shop
80,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Dairy Queen,43.710378,-79.290701,Ice Cream Shop
81,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Warden & St. Clair,43.712057,-79.281005,Intersection
82,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,TTC Bus #68 Warden,43.711778,-79.279714,Bus Line
83,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,TTC Bus 102 Markham Road,43.711381,-79.279588,Bus Line
84,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Warden Station Bus Loop,43.711241,-79.279576,Bus Station
85,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Cafe on the go,43.711151,-79.279469,Bakery
86,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Warden Subway Station,43.711238,-79.279444,Metro Station
87,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Warden Woods Park,43.710527,-79.278966,Park
88,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Clairlea Futbol Centre,43.715234,-79.286506,Soccer Field


In [38]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",2,2,2,2,2,2
Birch Cliff,4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Cliffcrest, Cliffside",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",8,8,8,8,8,8
"Highland Creek, Rouge Hill, Port Union",1,1,1,1,1,1
"Ionview, Kennedy Park",5,5,5,5,5,5


In [39]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 56 uniques categories.


In [37]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Hookah Bar,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Motel,Moving Target,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Steeles West,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Steeles West,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Steeles West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Steeles West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,Steeles West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [38]:
scarb_onehot.shape

(89, 60)

In [39]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Hookah Bar,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Motel,Moving Target,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.25,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.25,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.25,0.000000,0.000000,0.0,0.000000
1,"Agincourt North, Milliken",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.500000,0.000000,0.0,0.0,0.5,0.00,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000
2,Birch Cliff,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.25,0.000000,0.000000,0.0,0.000000
3,Cedarbrae,0.000000,0.142857,0.0,0.0,0.142857,0.142857,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.142857,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.142857,0.000000,0.00,0.0,0.142857,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.142857,0.0,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.000000,0.000000,0.0,0.0,0.111111,0.000000,0.0,0.00,0.000000,0.222222,0.111111,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.111111,0.000000,0.111111,0.0,0.0,0.0,0.000000,0.00,0.0,0.111111,0.0,0.000000,0.0,0.0,0.0,0.111111,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.111111,0.000000,0.0,0.000000
5,"Cliffcrest, Cliffside",0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.333333,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000
6,"Dorset Park, Scarborough Town Centre, Wexford ...",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00,0.111111,0.000000,0.000000,0.00,0.000000,0.111111,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.000000,0.111111,0.00,0.0,0.000000,0.0,0.0,0.000000,0.222222,0.000000,0.0,0.0,0.0,0.111111,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.111111,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.111111


### Get top 10 venues per neighborhood

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North, Milliken",Park,Playground,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,Birch Cliff,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Grocery Store,General Entertainment
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Ice Cream Shop,Bus Station,Park,Soccer Field,Intersection,Metro Station,Construction & Landscaping,Cosmetics Shop
5,"Cliffcrest, Cliffside",American Restaurant,Motel,Health & Beauty Service,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Latin American Restaurant,Light Rail Station,Chinese Restaurant,Thrift / Vintage Store,Bakery,Bank,Fried Chicken Joint
7,"Highland Creek, Rouge Hill, Port Union",Bar,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
8,"Ionview, Kennedy Park",Coffee Shop,Discount Store,Department Store,Chinese Restaurant,Train Station,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint
9,"Maryvale, Wexford",Sandwich Place,Auto Garage,Smoke Shop,Shopping Mall,Breakfast Spot,Discount Store,College Stadium,Construction & Landscaping,Cosmetics Shop,Department Store


### Run k-means to cluster the neighborhoods into 5 clusters

In [46]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(16)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([0, 4, 0, 0, 0, 0, 0, 3, 0, 0])

### Include kmeans.labels_ into the original Scarborough dataframe

In [52]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,M1W,Scarborough,Steeles West,43.799525,-79.318389,0,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Cosmetics Shop,Pharmacy,Pizza Place,Breakfast Spot,Grocery Store,Japanese Restaurant,Electronics Store
1,4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Playground,Construction & Landscaping,Health & Beauty Service,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,6,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Health & Beauty Service,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
3,10,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Bar,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,16,M1N,Scarborough,Birch Cliff,43.692657,-79.264848,0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,31,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849,0,Sandwich Place,Auto Garage,Smoke Shop,Shopping Mall,Breakfast Spot,Discount Store,College Stadium,Construction & Landscaping,Cosmetics Shop,Department Store
6,34,M1V,Scarborough,"Agincourt North, Milliken",43.815252,-79.284577,0,Park,Playground,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
7,36,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Grocery Store,General Entertainment
8,41,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302,0,Pizza Place,Noodle House,Pharmacy,Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Rental Car Location
9,49,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476,0,American Restaurant,Motel,Health & Beauty Service,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


### Visualize the clusters in the map

### Examine each of the five clusters

In [59]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1W,-79.318389,0,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Cosmetics Shop,Pharmacy,Pizza Place,Breakfast Spot,Grocery Store,Japanese Restaurant,Electronics Store
2,M1G,-79.216917,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Health & Beauty Service,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
3,M1C,-79.160497,0,Bar,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1N,-79.264848,0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,M1R,-79.295849,0,Sandwich Place,Auto Garage,Smoke Shop,Shopping Mall,Breakfast Spot,Discount Store,College Stadium,Construction & Landscaping,Cosmetics Shop,Department Store
6,M1V,-79.284577,0,Park,Playground,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
8,M1T,-79.304302,0,Pizza Place,Noodle House,Pharmacy,Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Rental Car Location
9,M1M,-79.239476,0,American Restaurant,Motel,Health & Beauty Service,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
10,M1E,-79.188711,0,BBQ Joint,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint
13,M1K,-79.262029,0,Coffee Shop,Discount Store,Department Store,Chinese Restaurant,Train Station,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint


In [60]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,M1X,-79.205636,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]    

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,M1B,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,Train Station,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


In [62]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M1H,-79.239476,3,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Grocery Store,General Entertainment


In [63]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1J,-79.239476,4,Playground,Construction & Landscaping,Health & Beauty Service,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
12,M1S,-79.262029,4,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
